### Code for model

In [1]:
from generator import RenderedTextGenerator
from model import *
import os
import torch
import copy
import time
import numpy as np
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from skimage import io, img_as_float
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
class DatasetGenerator(Dataset):
    def __init__(self, text_generator, transform=None, num_words : int = 8):
        super(DatasetGenerator, self).__init__()
        self.text_generator = text_generator
        self.transform = transform
        self.num_words = num_words
        
    def __len__(self):
        return 124
    
    def __getitem__(self, idx):
        imgs, labels = self.text_generator.render_text()
        
        if self.transform is not None:
            return torch.stack(
                [self.transform(image = imgs[i, :, :, :3])['image'].float() for i in range(self.num_words)]), torch.tensor(labels)
        
        return torch.tensor(imgs), torch.tensor(labels)
    
def worker_init_fn(worker_id):
    torch_seed = torch.initial_seed()
    if torch_seed >= 2**30:  # make sure torch_seed + workder_id < 2**32
        torch_seed = torch_seed % 2**30
    random.seed(torch_seed + worker_id)
    np.random.seed(torch_seed + worker_id)
    
    
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = torch.tensor([0.])
                    loss = loss.to(device)
                    for i in range(outputs.shape[0]):
                        loss += criterion(outputs[i], labels[i])#C
                        
                    _, preds = torch.max(outputs, 2)
                    #print(preds)
                        
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset) / 8

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                pass
                val_acc_history.append(epoch_acc.detach().cpu().clone().numpy())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    
    model.load_state_dict(best_model_wts)
    return model, val_acc_history    

In [3]:
rtg = RenderedTextGenerator(batch_size=1, fontsize_range=(10, 22), fonts_folder='fonts/', img_shape=(224, 224),
                            fonttype='bold')

input_size = 224
train_transform = A.Compose([
    A.ShiftScaleRotate(shift_limit = 0.05, scale_limit = 0.05, rotate_limit = 0.05, p = 0.5),
    A.SmallestMaxSize(max_size=input_size),
    #A.RandomCrop(height=input_size, width=input_size),
    A.RGBShift(r_shift_limit=0.7, g_shift_limit=0.7, b_shift_limit=0.7, p = 0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

dataset_gen = DatasetGenerator(rtg, train_transform)

val_transform = A.Compose([
        A.SmallestMaxSize(max_size=input_size),
        #A.CenterCrop(height=input_size, width=input_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
        ToTensorV2(),
])

val_dataset_gen = DatasetGenerator(rtg, val_transform)

train_dataloader = DataLoader(dataset_gen, batch_size=16,worker_init_fn=worker_init_fn)
val_dataloader = DataLoader(val_dataset_gen, batch_size=16, worker_init_fn=worker_init_fn)

dataloaders_dict = {'train' : train_dataloader,
                  'val': val_dataloader
}

model = BoldClassifier() 

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
model = model.to(device).float()
model, history= train_model(model,
                              dataloaders_dict, 
                              criterion,
                              optimizer,
                              num_epochs=20
)

Epoch 0/19
----------
train Loss: 13.7497 Acc: 0.6724
val Loss: 13.8393 Acc: 0.6593

Epoch 1/19
----------
train Loss: 5.0356 Acc: 0.8438
val Loss: 11.9902 Acc: 0.8377

Epoch 2/19
----------
train Loss: 4.2193 Acc: 0.8952
val Loss: 8.2223 Acc: 0.8871

Epoch 3/19
----------
train Loss: 3.0782 Acc: 0.9294
val Loss: 2.3109 Acc: 0.9415

Epoch 4/19
----------
train Loss: 2.4626 Acc: 0.9405
val Loss: 20.9275 Acc: 0.4778

Epoch 5/19
----------
train Loss: 2.0894 Acc: 0.9446
val Loss: 3.3251 Acc: 0.9163

Epoch 6/19
----------
train Loss: 2.4161 Acc: 0.9345
val Loss: 7.9205 Acc: 0.7933

Epoch 7/19
----------
train Loss: 2.6560 Acc: 0.9294
val Loss: 4.0781 Acc: 0.9153

Epoch 8/19
----------
train Loss: 2.6494 Acc: 0.9425
val Loss: 2.7649 Acc: 0.9415

Epoch 9/19
----------
train Loss: 2.2391 Acc: 0.9365
val Loss: 1.6715 Acc: 0.9536

Epoch 10/19
----------
train Loss: 3.6311 Acc: 0.9183
val Loss: 2.6948 Acc: 0.9073

Epoch 11/19
----------
train Loss: 2.4560 Acc: 0.9375
val Loss: 2.6235 Acc: 0.9264

In [9]:
'''
torch.save(model.state_dict(), 'model.pth')
model = BoldClassifier()
model.load_state_dict(torch.load('model.pth'))
model.eval()
model = model.to(device).float()
test_transform = A.Compose([
        A.SmallestMaxSize(max_size=input_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
])

test_dataset_gen = DatasetGenerator(rtg, test_transform)
test_dataloader = DataLoader(test_dataset_gen, batch_size=10, worker_init_fn=worker_init_fn)

for imgs, labels in test_dataloader:
    imgs = imgs.to(device)
    labels = labels.to(device)
    out = model(imgs)
    _, res = torch.max(out, dim = 2)
    print(torch.sum(res == labels).float() / (res.shape[0] * res.shape[1]))
''';